# Importações

In [4]:
import sys
path_to_module = 'c:/users/thiag/documents/ia_ufc/venv/lib/site-packages'
sys.path.append(path_to_module)
import cv2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras  import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Inicializando o gerador de imagens para fazer o pré-processamento

In [5]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [6]:
# pre-processamento das imagens de treino
train_generator = image_dataset_from_directory(
    'data2/',
    validation_split=0.2,
    subset="training",
    image_size=(48,48),
    batch_size=32,
    label_mode="categorical",
    seed = 42,
)
     

Found 392 files belonging to 2 classes.
Using 314 files for training.


In [7]:
# pre-processamento das imagens de treino
validation_generator =  image_dataset_from_directory(
    'data2/',
    validation_split=0.2,
    subset="validation",
    image_size=(48,48),
    batch_size=32,
    label_mode="categorical",
    seed = 42,
)

Found 392 files belonging to 2 classes.
Using 78 files for validation.


## Classes do modelo

In [8]:
classes = train_generator.class_names
classes

['Covid', 'Normal']

# Criando a estrutura do modelo

In [10]:
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

augmented_training = train_generator.map(lambda x , y:(data_augmentation(x, training = True), y))

augmented_training = augmented_training.prefetch(buffer_size=32)
validation_generator = validation_generator.prefetch(buffer_size=32)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
     

Definição do modelo sequencial. Inicialmente são aplicados nas imagens um layer de convolução, seguido de um layer de Pooling cujo máximo é extraído. Ao repertirmos o processo algumas vezes, obtemos matrizes que resumem as propriedades gerais das imagens para serem utilizadas na rede neural.

In [ ]:
emotion_model.summary()

In [ ]:
model.add(Flatten()) # Camada de entrada

model.add(Dense(64, activation="relu")) # Camada oculta

model.add(Dense(2, activation="softmax")) # Camada de saída, "softmax" é utilizada para saídas com muitas classes (array com multiplas probabilidades)

Aplainamento do modelo e aplicação dos nós densos da rede.

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]) # Compilação do modelo; hiperparametros

In [ ]:
epochs = 20
history = model.fit(augmented_training, epochs=epochs, validation_data=validation)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(emotion_model_info.history['accuracy'])
plt.plot(emotion_model_info.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(emotion_model_info.history['loss'])
plt.plot(emotion_model_info.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
emotion_model_info.history

# Salvando modelo gerado

In [ ]:
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')